In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 1. Create pseudo (synthetic) data
np.random.seed(42)
X = np.random.rand(100, 2)  # 100 samples, 2 features

# 2. Create labels (binary classification based on a simple rule)
# if x1 + x2 > 1 => label 1, else 0
y = (X[:, 0] + X[:, 1] > 1).astype(int)

# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Train a machine learning model
model = LogisticRegression()
model.fit(X_train, y_train)

# 5. Make predictions
predictions = model.predict(X_test)

# 6. Evaluate
accuracy = accuracy_score(y_test, predictions)

print("Predictions:", predictions)
print("True Labels:", y_test)
print("Accuracy:", accuracy)


Predictions: [0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1]
True Labels: [0 1 1 0 1 0 0 1 0 1 0 0 1 0 0 1 0 1 1 1]
Accuracy: 0.85


In [30]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 1. Data
texts = [
    "AI is amazing",
    "I hate this medicine",
    "The robot works perfectly",
    "This diet doesn't work",
    "Robotics is the future",
    "This product is terrible",
    "This surgery helped a lot",
    "I love machine learning"
]

# Labels
# Sentiment: 1 = positive, 0 = negative
# Topic:     1 = tech,     0 = health
sentiments = [1, 0, 1, 0, 1, 0, 1, 1]
topics     = [1, 0, 1, 0, 1, 0, 0, 1]

# 2. Vectorize
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

# 3. Train-test split
y = list(zip(sentiments, topics))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 4. Model
model = MultiOutputClassifier(LogisticRegression())
model.fit(X_train, y_train)

# 5. Predict
predictions = model.predict(X_test)
y_test_sentiments, y_test_topics = zip(*y_test)
pred_sentiments, pred_topics = zip(*predictions)

# 6. Evaluation
print("=== Sentiment Classification ===")
print(classification_report(y_test_sentiments, pred_sentiments, zero_division=0))

print("=== Topic Classification ===")
print(classification_report(y_test_topics, pred_topics, zero_division=0))

# 7. Predict on New Text
new_text = ["This robot is bad"]
X_new = vectorizer.transform(new_text)
pred_sentiment, pred_topic = model.predict(X_new)[0]

# 8. Decode labels
sentiment_map = {0: "Negative", 1: "Positive"}
topic_map = {0: "Health", 1: "Tech"}

print("\n=== New Text Prediction ===")
print("Text:", new_text[0])
print("Sentiment:", sentiment_map[pred_sentiment])
print("Topic    :", topic_map[pred_topic])


=== Sentiment Classification ===
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       2.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0

=== Topic Classification ===
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       2.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0


=== New Text Prediction ===
Text: This robot is bad
Sentiment: Positive
Topic    : Tech


In [25]:
# Your new input text
new_text = ["This robot is useless"]

# Vectorize it using the same vectorizer
X_new = vectorizer.transform(new_text)

# Predict using the trained multi-label model
pred_sentiment, pred_topic = model.predict(X_new)[0]

# Map numeric predictions back to labels
sentiment_map = {0: "Negative", 1: "Positive"}
topic_map = {0: "Health", 1: "Tech"}

print("Prediction:")
print("  Sentiment:", sentiment_map[pred_sentiment])
print("  Topic    :", topic_map[pred_topic])


Prediction:
  Sentiment: Positive
  Topic    : Tech


In [34]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

# 1. Data
texts = [
    "AI breakthroughs amaze me every day",
    "This treatment saved my life",
    "Finance laws are so confusing",
    "The concert last night was unforgettable",
    "This robot vacuum is a game changer",
    "The new fitness routine feels exhausting",
    "Healthcare policies need urgent reform",
    "Virtual reality made the game more immersive",
    "Banking apps should improve security",
    "This documentary about the cosmos blew my mind"
]

# Sentiment (10 classes)
sentiments = list(range(10))  # [0–9]

# Topic (10 classes)
topics = list(range(10))  # [0–9]

# 2. Text Vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts).toarray()

# 3. One-hot encode the labels
y_sentiment = to_categorical(sentiments, num_classes=10)
y_topic = to_categorical(topics, num_classes=10)

# 4. Train-test split
X_train, X_test, y_train_sent, y_test_sent, y_train_topic, y_test_topic = train_test_split(
    X, y_sentiment, y_topic, test_size=0.2, random_state=42
)

# 5. Build the model
input_dim = X_train.shape[1]

inputs = layers.Input(shape=(input_dim,))
x = layers.Dense(128, activation='relu')(inputs)
x = layers.Dropout(0.3)(x)

out_sentiment = layers.Dense(10, activation='softmax', name='sentiment')(x)
out_topic = layers.Dense(10, activation='softmax', name='topic')(x)

model = models.Model(inputs=inputs, outputs=[out_sentiment, out_topic])

model.compile(
    optimizer='adam',
    loss={'sentiment': 'categorical_crossentropy', 'topic': 'categorical_crossentropy'},
    metrics={'sentiment': 'accuracy', 'topic': 'accuracy'}
)

# 6. Train
model.fit(
    X_train,
    {'sentiment': y_train_sent, 'topic': y_train_topic},
    epochs=40,
    verbose=1
)

# 7. Predict
pred_sent, pred_topic = model.predict(X_test)
pred_sent_labels = np.argmax(pred_sent, axis=1)
pred_topic_labels = np.argmax(pred_topic, axis=1)

true_sent_labels = np.argmax(y_test_sent, axis=1)
true_topic_labels = np.argmax(y_test_topic, axis=1)

# 8. Evaluate
print("\n=== Sentiment Classification ===")
print(classification_report(true_sent_labels, pred_sent_labels, zero_division=0))

print("=== Topic Classification ===")
print(classification_report(true_topic_labels, pred_topic_labels, zero_division=0))

# 9. Predict new sample
new_text = ["I love the future of robotics and automation"]
print(new_text)
X_new = vectorizer.transform(new_text).toarray()
new_pred_sent, new_pred_topic = model.predict(X_new)

print("\n=== New Prediction ===")
print("Sentiment class:", print(np.argmax(new_pred_sent[0])))
print("Topic class    :", np.argmax(new_pred_topic[0]))


Epoch 1/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 4.5961 - sentiment_accuracy: 0.2500 - sentiment_loss: 2.3229 - topic_accuracy: 0.0000e+00 - topic_loss: 2.2732
Epoch 2/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 4.5968 - sentiment_accuracy: 0.2500 - sentiment_loss: 2.2936 - topic_accuracy: 0.0000e+00 - topic_loss: 2.3032
Epoch 3/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 4.5900 - sentiment_accuracy: 0.1250 - sentiment_loss: 2.3177 - topic_accuracy: 0.1250 - topic_loss: 2.2722
Epoch 4/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - loss: 4.5396 - sentiment_accuracy: 0.3750 - sentiment_loss: 2.2726 - topic_accuracy: 0.0000e+00 - topic_loss: 2.2670
Epoch 5/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 4.5075 - sentiment_accuracy: 0.0000e+00 - sentiment_loss: 2.2718 - topic_accuracy: 0.2500 - topic_loss: 2.2357
Epoch 6/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 4.4226 - sentiment_accuracy: 0.3750 - sentiment_loss: 2.2360 - topic_accuracy: 0.3750 - topic_loss: 2.1866